In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\programdata\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201104


毎週共有されるCSCからのセキュポスデータ（新規入会）を取り込み、Datastudio読込用に編集する

In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [6]:
#ディレクトリ指定
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\承認データ")

#input
input_file = "承認データ.xlsx"

#抽出する日付指定
yyyymmdd = 20201027

#out_put
output_file = "承認データdatastudio読込用_new"

In [7]:
print(yyyymmdd)

20201027


新規Vintage CUSTデータの読込＆集計

In [10]:
%%time
df = pd.read_excel(data_dir /f"{input_file}",
                  dtype={'カード加入日':int,
                        'カード提携会社1CD':str,
                        'カード提携会社2CD':str,
                        '切替CD':str,
                        '提携カード特殊エリア':str,
                        '受付部支店':str,
                        'リボ支払方法CD':str,
                        'エンボス内容':str,
                        'カード種類CD':str,
                        '入会申込番号':str,
                        '提携先顧客番号1':str},encoding='shift-jis')
df = df[df['カード加入日'] >= yyyymmdd]

Wall time: 9.05 s


In [11]:
df.head()

,カード加入日,続柄,カード提携会社1CD,カード提携会社2CD,カード種類CD,カードブランドCD,切替CD,勧誘CD,受付部支店,再発行CD,旧カード種類CD,旧カードブランドCD,リボ支払方法CD,勧誘団体CD,提携カード特殊エリア,エンボス内容,カード年会費CD,入会申込番号,提携先顧客番号1
47545,20201027,0,D00,04,31,2,0,3,299,0,NaN,NaN,12,2000,NaN,02000,2,155051922063843,NaN
47546,20201027,0,D00,01,31,1,0,3,299,0,NaN,NaN,52,2000,NaN,02000,2,155051922063249,NaN
47547,20201027,0,D00,50,31,0,0,8,103,0,NaN,NaN,12,3000,20,03000343237,1,155203048057783,NaN
47548,20201027,0,D00,50,31,0,0,8,104,0,NaN,NaN,12,4000,20,04000,1,155203038056001,NaN
47549,20201027,0,D00,01,31,1,0,3,299,0,NaN,NaN,12,6000,NaN,06000,2,155051922062548,NaN


In [12]:
print(df.dtypes)

カード加入日          int32
続柄              int64
カード提携会社1CD     object
カード提携会社2CD     object
カード種類CD        object
カードブランドCD       int64
切替CD           object
勧誘CD            int64
受付部支店          object
再発行CD           int64
旧カード種類CD      float64
旧カードブランドCD    float64
リボ支払方法CD       object
勧誘団体CD          int64
提携カード特殊エリア     object
エンボス内容         object
カード年会費CD        int64
入会申込番号         object
提携先顧客番号1       object
dtype: object


In [13]:
df = df.rename(columns={'カード加入日':'kanyu_date',
                        '続柄':'zokugara',
                        'カード提携会社1CD':'teikei_1',
                        'カード提携会社2CD':'teikei_2',
                        '切替CD':'kirikae',
                        '提携カード特殊エリア':'tokushu',
                        '受付部支店':'uketuke',
                        'カード種類CD':'card',
                        '入会申込番号':'applino',
                        '提携先顧客番号1':'campaign_cd'})
df.head()

,kanyu_date,zokugara,teikei_1,teikei_2,card,カードブランドCD,kirikae,勧誘CD,uketuke,再発行CD,旧カード種類CD,旧カードブランドCD,リボ支払方法CD,勧誘団体CD,tokushu,エンボス内容,カード年会費CD,applino,campaign_cd
47545,20201027,0,D00,04,31,2,0,3,299,0,NaN,NaN,12,2000,NaN,02000,2,155051922063843,NaN
47546,20201027,0,D00,01,31,1,0,3,299,0,NaN,NaN,52,2000,NaN,02000,2,155051922063249,NaN
47547,20201027,0,D00,50,31,0,0,8,103,0,NaN,NaN,12,3000,20,03000343237,1,155203048057783,NaN
47548,20201027,0,D00,50,31,0,0,8,104,0,NaN,NaN,12,4000,20,04000,1,155203038056001,NaN
47549,20201027,0,D00,01,31,1,0,3,299,0,NaN,NaN,12,6000,NaN,06000,2,155051922062548,NaN


In [25]:
#print(df['campaign_cd'].unique())

In [14]:
q = """
        SELECT
        kanyu_date
        ,case 
            when substr(tokushu,1,2) = "20" then '店頭タブレット'
            when uketuke = "299" and substr(campaign_cd,1,2) = "10" then 'オンライン(百貨店送客)'
            when uketuke = "299" then 'オンライン'
            when uketuke in ("200","001") then '郵送'
            else '店頭紙' 
            end as uketsuke_channel
        ,case 
            when kirikae = "0" then 'N/A'
            when substr(tokushu,1,2) = "80" then 'DM'
            when substr(tokushu,1,2) = "81" then "コンタクト"
            when substr(tokushu,1,2) = "86" then "店頭"
            else "店頭"
            end as kirikae_channel
        ,case
            when teikei_2 =50 then '臨時'
            when teikei_1 ="D02" then '外商'
            when teikei_2 ="05" then 'ハウスゴールド'
            when card ="02"  then 'ゴールド'
            else '一般'
            end as card_shubetsu
        ,case 
            when zokugara = 0 then '本人'
            else '家族' 
            end as kazoku_shubetsu
        ,sum(case when kirikae = "0" then 1 else 0 end ) as no_of_new
        ,sum(case when kirikae = "1" then 1 else 0 end ) as no_of_kirikae
        FROM 
          df
        where
          teikei_2 not in ("95") 
        group by
        1,2,3,4,5
        """

In [15]:
df_sql = sqldf(q, locals())
df_sql['kanyu_date']=pd.to_datetime(df_sql['kanyu_date'], format='%Y%m%d')
df_sql['kanyu_year'] = df_sql['kanyu_date'].dt.year
df_sql['kanyu_month'] = df_sql['kanyu_date'].dt.month
df_sql['kanyu_day'] = df_sql['kanyu_date'].dt.day
df_sql['kanyu_date'] = df_sql['kanyu_date'].dt.strftime('%Y/%m/%d')
df_sql.head()

,kanyu_date,uketsuke_channel,kirikae_channel,card_shubetsu,kazoku_shubetsu,no_of_new,no_of_kirikae,kanyu_year,kanyu_month,kanyu_day
0,2020/10/27,オンライン,N/A,ゴールド,本人,1,0,2020,10,27
1,2020/10/27,オンライン,N/A,一般,本人,89,0,2020,10,27
2,2020/10/27,オンライン(百貨店送客),N/A,一般,本人,2,0,2020,10,27
3,2020/10/27,店頭タブレット,N/A,ゴールド,家族,2,0,2020,10,27
4,2020/10/27,店頭タブレット,N/A,ゴールド,本人,8,0,2020,10,27


CSVアウトプット

In [16]:
df_sql.to_csv(data_dir /f"{output_file}.csv", index=None,header=True,encoding='utf-8-sig')